In [1]:
import sys
import os

if 'root_dir' not in globals():
    # rootディレクトリへのパスを設定
    root_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
    os.chdir(root_dir)



In [11]:
query = 'オーガニック'

# 記事を取得する

In [ ]:
# 自作したコードをimportする
from my_codes import fetch_articles


In [ ]:
query_keys_all_df, selected_all_user_data = fetch_articles.main(
    query = query, size=100, batches=10000,interval=1
)

# 前処理をする

In [37]:
from my_codes import preprocessing

In [ ]:
preprocessing.main(query=query)

# データベースに保存する

In [15]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Text, Boolean, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# データベースエンジンの作成（例：SQLite）
engine = create_engine('sqlite:///data/user_all_post.db', echo=True)

# Base クラスの作成
Base = declarative_base()

# Notes クラスの定義
class Notes(Base):
    __tablename__ = 'notes'
    
    id = Column(Integer)
    user_id = Column(Integer)
    status = Column(Text)
    type = Column(Text)
    key = Column(String, primary_key=True, unique=True)
    slug = Column(Text)
    name = Column(Text)
    body = Column(Text)
    created_at = Column(DateTime)
    can_read = Column(Boolean)
    user_key = Column(Text)
    urlname = Column(Text)
    nickname = Column(Text)
    note_count = Column(Integer)
    user_created_at = Column(DateTime)
    tokenized_body = Column(Text)

# テーブルの作成
Base.metadata.create_all(engine, checkfirst=True)

# セッションの作成
Session = sessionmaker(bind=engine)
session = Session()

2024-06-02 18:55:09,321 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-02 18:55:09,321 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("notes")
2024-06-02 18:55:09,322 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-02 18:55:09,322 INFO sqlalchemy.engine.Engine COMMIT


/var/folders/xq/9pq38rcj7l179qz2bf_8ms5m0000gn/T/ipykernel_15313/1348950832.py:10: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [20]:
# ORMを使って特定のカラムを取得
database_keys = session.query(Notes.key).all()
session.close()
# リストをDataFrameに変換
df_database = pd.DataFrame(database_keys, columns=['key'])

2024-06-02 18:55:59,179 INFO sqlalchemy.engine.Engine SELECT notes."key" AS notes_key 
FROM notes
2024-06-02 18:55:59,180 INFO sqlalchemy.engine.Engine [cached since 44.61s ago] ()
2024-06-02 18:55:59,334 INFO sqlalchemy.engine.Engine ROLLBACK


In [17]:
# CSV ファイルのパス
csv_file_path = f'data/{query}/{query}_data_preprocessed.csv'
# CSV ファイルの読み込み
df = pd.read_csv(csv_file_path, index_col=0)

In [22]:
# データベースに存在しないデータを取得
new_df = df[~df['key'].isin(df_database['key'])]

# 新しいデータをデータベースに挿入
if not new_df.empty:
    new_df.to_sql('notes', con=engine, if_exists='append', index=False)
else:
    print("新しいデータはありません。")

新しいデータはありません。
